In [1]:
!pip install pytorch_pretrained_bert pytorch-nlp

    100% |████████████████████████████████| 133kB 6.5MB/s 
    100% |████████████████████████████████| 92kB 25.4MB/s 
    100% |████████████████████████████████| 133kB 15.9MB/s 
    100% |████████████████████████████████| 655kB 21.0MB/s 
    100% |████████████████████████████████| 5.7MB 6.5MB/s 
    100% |████████████████████████████████| 71kB 24.1MB/s 
    100% |████████████████████████████████| 552kB 29.0MB/s 
  Running setup.py bdist_wheel for regex ... done
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built regex
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 3.0.2 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.4.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.2.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading 

In [2]:
import sys
import numpy as np
import random as rn
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from torchnlp.datasets import imdb_dataset
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output

Using TensorFlow backend.


In [3]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

## Prepare the Data

In [4]:
train_data, test_data = imdb_dataset(train=True, test=True)
rn.shuffle(train_data)
rn.shuffle(test_data)
train_data = train_data[:1000]
test_data = test_data[:100]

aclImdb_v1.tar.gz: 84.1MB [00:02, 39.5MB/s]                            


In [5]:
len(train_data)

1000

In [6]:
train_data[0]

{'text': "Taking old collection of stories poses a challenge for the production team, how can this classic character be brought up to date and make it interesting enough to capture a new audience while stirring memories from her former audience. In my opinion, their mission was accomplished. A must see for young children, pre-teens, teens, and their parents. OK there are a couple scary moments that are resolved in short order, but parents with young children should sit tight, the movie moves on to better things. I am going to go with those astute user reviewers who point out that Emma Roberts provides us with a positive role model for young women, not syrupy about it either. Nancy balances her femininity with career minded sleuthing skills. A lot to like in here, laughs, doesn't take itself too seriously, a real mystery to solve (one you can figure out yourself as an added bonus, and likable characters. Nancy Drew even makes a good date movie in my opinion.",
 'sentiment': 'pos'}

In [7]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(1000, 1000, 100, 100)

In [8]:
type(train_texts), type(train_texts[0])

(tuple, str)

In [9]:
train_texts[0]

"Taking old collection of stories poses a challenge for the production team, how can this classic character be brought up to date and make it interesting enough to capture a new audience while stirring memories from her former audience. In my opinion, their mission was accomplished. A must see for young children, pre-teens, teens, and their parents. OK there are a couple scary moments that are resolved in short order, but parents with young children should sit tight, the movie moves on to better things. I am going to go with those astute user reviewers who point out that Emma Roberts provides us with a positive role model for young women, not syrupy about it either. Nancy balances her femininity with career minded sleuthing skills. A lot to like in here, laughs, doesn't take itself too seriously, a real mystery to solve (one you can figure out yourself as an added bonus, and likable characters. Nancy Drew even makes a good date movie in my opinion."

In [61]:
train_texts[1]

"A sober, reflexive piece, a little miniature which blossoms into a magnificent humane pictorial sequence which goes beyond a mere dramatization for the screen. This quiet little story will hold you enthralled - if you do not have too many problems with the various Spanish accents ranging from Mexican to Peruvian, and Marisa Paredes' more authentic Iberian Peninsular usage! Garcíadiego has accomplished a perfect adaptation from the novel: even the grand maestro García Márquez should be proud of her superb work. And hats off to Arturo Ripstein who has so ably concerted the whole effort into a gem, a ruby, and so refined, so elegant, so sensitive, so touchingly.....<br /><br />El Coronel - Fernando Luján - is waiting to get his pension, while he continues to live in his ramshackle timber dwelling deep in the Colombian jungle (however, filmed elsewhere, NOT in Colombia) with his fighting cock and his wife (in that order?). And that is all there is to it.<br /><br />But, oh, so much more..

In [10]:
train_labels[0]

'pos'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 1014489.88B/s]


In [12]:
tokenizer.tokenize('Hi my name is Dima')

['hi', 'my', 'name', 'is', 'dim', '##a']

In [13]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)                   
                   

(1000, 100)

In [14]:
type(train_tokens), type(train_tokens[0])

(list, list)

In [15]:
train_tokens[0]

['[CLS]',
 'taking',
 'old',
 'collection',
 'of',
 'stories',
 'poses',
 'a',
 'challenge',
 'for',
 'the',
 'production',
 'team',
 ',',
 'how',
 'can',
 'this',
 'classic',
 'character',
 'be',
 'brought',
 'up',
 'to',
 'date',
 'and',
 'make',
 'it',
 'interesting',
 'enough',
 'to',
 'capture',
 'a',
 'new',
 'audience',
 'while',
 'stirring',
 'memories',
 'from',
 'her',
 'former',
 'audience',
 '.',
 'in',
 'my',
 'opinion',
 ',',
 'their',
 'mission',
 'was',
 'accomplished',
 '.',
 'a',
 'must',
 'see',
 'for',
 'young',
 'children',
 ',',
 'pre',
 '-',
 'teens',
 ',',
 'teens',
 ',',
 'and',
 'their',
 'parents',
 '.',
 'ok',
 'there',
 'are',
 'a',
 'couple',
 'scary',
 'moments',
 'that',
 'are',
 'resolved',
 'in',
 'short',
 'order',
 ',',
 'but',
 'parents',
 'with',
 'young',
 'children',
 'should',
 'sit',
 'tight',
 ',',
 'the',
 'movie',
 'moves',
 'on',
 'to',
 'better',
 'things',
 '.',
 'i',
 'am',
 'going',
 'to',
 'go',
 'with',
 'those',
 'as',
 '##tute',
 'u

In [16]:
len(train_tokens[0])

202

In [17]:
rev_lengths = [len(item) for item in train_tokens]

In [18]:
len(rev_lengths)

1000

In [19]:
import pandas as pd
rev_lengths = pd.DataFrame(rev_lengths)

In [20]:
rev_lengths.describe()

,0
count,1000.000000
mean,281.615000
std,140.904176
min,15.000000
25%,168.000000
50%,242.500000
75%,404.250000
max,512.000000


In [21]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((1000, 512), (100, 512))

In [22]:
train_tokens_ids[0]

array([  101,  2635,  2214,  3074,  1997,  3441, 22382,  1037,  4119,
        2005,  1996,  2537,  2136,  1010,  2129,  2064,  2023,  4438,
        2839,  2022,  2716,  2039,  2000,  3058,  1998,  2191,  2009,
        5875,  2438,  2000,  5425,  1037,  2047,  4378,  2096, 18385,
        5758,  2013,  2014,  2280,  4378,  1012,  1999,  2026,  5448,
        1010,  2037,  3260,  2001,  8885,  1012,  1037,  2442,  2156,
        2005,  2402,  2336,  1010,  3653,  1011, 13496,  1010, 13496,
        1010,  1998,  2037,  3008,  1012,  7929,  2045,  2024,  1037,
        3232, 12459,  5312,  2008,  2024, 10395,  1999,  2460,  2344,
        1010,  2021,  3008,  2007,  2402,  2336,  2323,  4133,  4389,
        1010,  1996,  3185,  5829,  2006,  2000,  2488,  2477,  1012,
        1045,  2572,  2183,  2000,  2175,  2007,  2216,  2004, 24518,
        5310, 15814,  2040,  2391,  2041,  2008,  5616,  7031,  3640,
        2149,  2007,  1037,  3893,  2535,  2944,  2005,  2402,  2308,
        1010,  2025,

In [24]:
train_tokens_ids[0].shape, train_tokens_ids.shape

((512,), (1000, 512))

In [25]:
train_y = np.array(train_labels) == 'pos'
test_y = np.array(test_labels) == 'pos'
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((1000,), (100,), 0.489, 0.5)

In [26]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [27]:
len(train_masks)

1000

In [28]:
train_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

# Baseline

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [30]:
baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [31]:
baseline_predicted = baseline_model.predict(test_texts)

In [32]:
print(classification_report(test_labels, baseline_predicted))

              precision    recall  f1-score   support

         neg       0.76      0.74      0.75        50
         pos       0.75      0.76      0.75        50

   micro avg       0.75      0.75      0.75       100
   macro avg       0.75      0.75      0.75       100
weighted avg       0.75      0.75      0.75       100



# Bert Model

In [44]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
        

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [46]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'0.106496M'

In [47]:
bert_clf = BertBinaryClassifier()
#bert_clf = bert_clf.cuda()


In [48]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'0.106496M'

In [56]:
x

tensor([[  101,  2635,  2214,  ...,     0,     0,     0],
        [  101,  4787, 14255,  ...,     0,     0,     0],
        [  101,  2641,  2011,  ...,     0,     0,     0]], device='cuda:0')

In [61]:
x = torch.tensor(train_tokens_ids[:3])#.to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [62]:
y = bert_clf(x)
y.cpu().detach().numpy()

array([[0.55415684],
       [0.5256586 ],
       [0.52362305]], dtype=float32)

In [63]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'0.188416M'

In [64]:
y, x, pooled = None, None, None
#torch.cuda.empty_cache()
#str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

# Fine-tune BERT

In [65]:
BATCH_SIZE = 4
EPOCHS = 10

In [66]:
train_y.shape

(1000,)

In [67]:
train_y.reshape(-1,1).shape

(1000, 1)

In [68]:
torch.tensor(train_y.reshape(-1,1))

TypeError: can't convert np.ndarray of type numpy.bool_. The only supported types are: double, float, float16, int64, int32, and uint8.

In [47]:
torch.tensor(train_y.reshape(-1,1)).float()

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
      

In [48]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

#str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [49]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)


In [50]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [51]:
param_optimizer

[]

In [52]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [ ]:
 torch.cuda.empty_cache()

In [53]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        #print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
        

Epoch:  1
23/250.0 loss: 0.6890591283639272 


KeyboardInterrupt: 

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
    

In [33]:
np.mean(bert_predicted)

0.56

In [34]:
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       0.86      0.76      0.81        50
        True       0.79      0.88      0.83        50

    accuracy                           0.82       100
   macro avg       0.82      0.82      0.82       100
weighted avg       0.82      0.82      0.82       100



Below are some cells where I try to manually break apart the code and see how the steps of embedding with BERT are done

In [34]:
model = BertModel.from_pretrained('bert-base-uncased')

100%|██████████| 407873900/407873900 [00:25<00:00, 15894993.27B/s]


In [72]:
linear = nn.Linear(768,1)

In [56]:
x = next(iter(train_dataloader))

In [59]:
x

[tensor([[ 101, 2028, 1011,  ...,    0,    0,    0],
         [ 101, 2028, 1997,  ...,    0,    0,    0],
         [ 101, 2893, 2439,  ...,    0,    0,    0],
         [ 101, 2887, 3419,  ..., 2005, 1996,  102]]),
 tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]]),
 tensor([[0.],
         [1.],
         [0.],
         [1.]])]

In [63]:
x[0].shape, x[1].shape, x[2].shape

(torch.Size([4, 512]), torch.Size([4, 512]), torch.Size([4, 1]))

In [87]:
x[0][0]

tensor([  101,  2028,  1011,  3602,  4038,  2008,  2763,  4520,  2715,  2154,
        10469,  2015,  1005,  4091,  2006,  3341,  1012,  2533,  3573,  7805,
        22396,  7867,  2003,  1999,  2293,  2007,  1996,  2801,  1997, 10834,
         1998,  3510,  1010, 22866,  2014,  8069,  2006,  2308,  1005,  1055,
         7298,  2127,  2016, 16794,  3565,  1011, 17395,  1999,  1996,  2433,
         1997, 21185,  5065, 20533,  3946,  1012,  1996,  2717,  1997,  1996,
         2143, 20228, 20620,  2013,  2028, 20228,  6977,  2000,  1996,  2279,
         2004,  1996, 21660,  7867,  7323,  2015,  2014, 12907,  1012,  1045,
         3984,  1996,  2773,  1000,  2566,  4801,  1000,  2074,  2055,  7680,
         1005,  1055,  2039,  7867,  1005,  1055,  3921,  2000,  1996,  2535,
         1012,  2016,  2515,  2031,  1037, 11951,  2868,  1010,  2021,  2044,
         2322,  2781,  1997, 24443, 21885,  2075,  2014, 21418, 14900,  1010,
         1045,  2211,  2000, 26641,  1012,  3946,  1005,  1055, 

In [66]:
y, pooled = model(x[0].long(), x[1].long(), output_all_encoded_layers=False)

In [68]:
y.shape

torch.Size([4, 512, 768])

In [69]:
pooled.shape

torch.Size([4, 768])

In [70]:
pooled[0].shape

torch.Size([768])

In [71]:
pooled[0]

tensor([ 6.2035e-01, -1.2935e-01, -6.3447e-01, -3.8974e-01,  4.2116e-01,
        -5.5024e-02, -9.3580e-01,  1.4767e-01, -5.9223e-01, -4.6779e-01,
         2.5108e-01,  3.9842e-01,  3.1596e-01,  5.0302e-01, -3.0484e-01,
         7.5943e-01,  8.6045e-01, -3.5990e-02,  6.9738e-03,  9.6940e-01,
         3.3081e-01,  9.9329e-01,  1.9115e-02,  3.4884e-01,  2.6564e-01,
         4.4706e-01,  2.5722e-02, -2.3618e-02, -1.0260e-01,  4.6844e-01,
         6.2713e-01, -3.7711e-02, -6.6075e-01, -2.5164e-01, -8.9383e-01,
        -2.2646e-01, -1.0696e-02,  4.0202e-01, -7.4280e-02,  3.4344e-01,
         2.4658e-01, -2.4652e-02,  8.9589e-01,  6.0270e-01, -3.4519e-02,
         1.0284e-01,  9.0841e-01,  1.4352e-01,  4.3273e-01,  7.6508e-01,
         4.6201e-01,  8.9849e-01, -1.2410e-01, -1.2699e-01, -5.5505e-02,
        -3.2303e-01, -2.6756e-01,  9.5763e-03, -4.8406e-02,  4.9331e-03,
        -3.1580e-01,  2.6484e-01, -6.0025e-01,  5.4551e-01,  1.9510e-01,
         7.8348e-01, -8.1185e-02,  4.8980e-02,  1.1

In [73]:
y[0].shape

torch.Size([512, 768])

In [74]:
z = linear(pooled)

In [75]:
z.shape

torch.Size([4, 1])

In [76]:
z[0]

tensor([-0.2232], grad_fn=<SelectBackward>)

In [79]:
sig = nn.Sigmoid()

In [82]:
prob = sig(z)

In [83]:
prob

tensor([[0.4444],
        [0.2773],
        [0.5422],
        [0.6100]], grad_fn=<SigmoidBackward>)